In [1]:
import os
import random
import numpy as np

import functools

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchtext import datasets
from torchtext.data import Field
from torchtext.data import BucketIterator


SEED = 241

In [2]:
def seed_everything(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)

  if torch.cuda.is_available(): 
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [3]:
def preprocessing(x):
  if len(x) > 1 and isinstance(x, list):
    if ' ' in x[0]:
      x = ' . '.join(x)
    else:
      x = ' '.join(x)
    x = x.split()
  return x

In [4]:
TEXT = Field(lower=True,
             use_vocab=True, 
             sequential=True, 
             init_token='<sos>',
             eos_token='<eos>',
             batch_first=True,
             preprocessing=preprocessing,
             include_lengths=True)

In [5]:
train_data, valid_data, test_data = datasets.BABI20.splits(text_field=TEXT)

tasks_1-20_v1-2.tar.gz:   4%|▍         | 606k/15.7M [00:00<00:02, 6.02MB/s]

downloading tasks_1-20_v1-2.tar.gz


tasks_1-20_v1-2.tar.gz: 100%|██████████| 15.7M/15.7M [00:01<00:00, 11.3MB/s]


In [6]:
TEXT.build_vocab(train_data)

In [7]:
index = 4

print(train_data.examples[index].query) # question
print(train_data.examples[index].answer) # answer
print(train_data.examples[index].story) # context

['where', 'is', 'sandra']
['bathroom']
['mary', 'moved', 'to', 'the', 'bathroom', '.', 'john', 'went', 'to', 'the', 'hallway', '.', 'daniel', 'went', 'back', 'to', 'the', 'hallway', '.', 'sandra', 'moved', 'to', 'the', 'garden', '.', 'john', 'moved', 'to', 'the', 'office', '.', 'sandra', 'journeyed', 'to', 'the', 'bathroom', '.', 'mary', 'moved', 'to', 'the', 'hallway', '.', 'daniel', 'travelled', 'to', 'the', 'office', '.', 'john', 'went', 'back', 'to', 'the', 'garden', '.', 'john', 'moved', 'to', 'the', 'bedroom']


In [8]:
def answer2span(batch_answer, batch_context):
  result = []
  candidates = []
  batch_answer = batch_answer[:, 1:]
  batch_answer = batch_answer[:, :-1]
  bs = len(batch_answer)
  for batch_num in range(bs):
    candidates.append([])
    answer = batch_answer[batch_num]
    context = batch_context[batch_num]
    for i in range(len(context)):
      ctx_token = context[i]
      if answer[0] == ctx_token:
        if np.allclose(answer, context[i: i + len(answer)]):
          start = i
          end = start + len(answer)
          candidates[batch_num].append((start, end))
    start, end = candidates[batch_num][-1]
    result.append([start, end])
  return result

In [9]:
batch_size = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


train_iterator = BucketIterator.splits((train_data,), batch_size=batch_size, device=device)[0]
valid_iterator = BucketIterator.splits((valid_data,), batch_size=batch_size, device=device)[0]
test_iterator = BucketIterator.splits((test_data,), batch_size=batch_size, device=device)[0]

![DrQa](https://github.com/kushalj001/pytorch-question-answering/blob/master/images/drqaflow.PNG?raw=true)

In [10]:
class AlignParagraphLayer(nn.Module):

  def __init__(self, emb_size):
    super().__init__()
    self.linear = nn.Linear(emb_size, emb_size)
    self.relu = nn.ReLU()

  def forward(self, context_emb, question_emb, question_mask):    
    # context_emb [bs, seq_len1, emb_dim]
    # question_emb [bs, seq_len2, emb_dim]
    # question_mask - padding mask [bs, seq_len2]

    context = self.linear(context_emb)
    context = self.relu(context)
    # context_emb [bs, seq_len1, emb_size]

    question = self.linear(question_emb)
    question = self.relu(question)
    # question_emb [bs, seq_len2, emb_size]

    question_T = question.permute(0, 2, 1)
    # question_emb_T [bs, emb_size, seq_len2]

    align_score = torch.bmm(context, question_T)
    # align_score [bs, seq_len1, seq_len2]

    qtn_mask = question_mask.unsqueeze(1).repeat(1, context_emb.size(1), 1)
    # qtn_mask [bs, seq_len1, seq_len2]

    align_score = align_score.masked_fill(qtn_mask == 0, -float('inf'))
    # align_score [bs, seq_len1, seq_len2]

    align_scores_flat = align_score.view(-1, question_T.size(-1))
    # align_scores_flat [bs * seq_len1, seq_len2]

    alphas = F.softmax(align_scores_flat, dim=1)
    # alpha [bs * seq_len1, seq_len2]

    alphas = alphas.view(-1, align_score.size(1), align_score.size(2))
    # alphas [bs, seq_len1, seq_len2]

    alphas_weighted_question = torch.bmm(alphas, question_emb)
    # alphas_weighted [bs, seq_len1, emb_dim]

    return alphas_weighted_question

In [11]:
class StackedBiLSTM(nn.Module):

  def __init__(self, emb_dim, hidden_dim, num_layers, dropout):
    super().__init__()
    self.num_layers = num_layers
    self.dropout = dropout
    self.rnns = nn.ModuleList([nn.LSTM(emb_dim, hidden_dim, num_layers=1, batch_first=True, bidirectional=True) for _ in range(num_layers)])
    
  def forward(self, x, x_lens):
    output = [x]
    hiddens = []
    cells = []
    for layer in self.rnns:
      x_packed = nn.utils.rnn.pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
      if hiddens:
        packed_layer_out, (hidden, cell) = layer(x_packed, (hidden, cell))
      else:
        packed_layer_out, (hidden, cell) = layer(x_packed)
      layer_out, _ = nn.utils.rnn.pad_packed_sequence(packed_layer_out, batch_first=True)
      layer_out = F.dropout(layer_out, p=self.dropout)
      # layer_out [bs, seq_len, hidden_dim]
      output.append(layer_out)
      hiddens.append(hidden)
      cells.append(cell)

    final_hidden = torch.cat(output[1:], dim=2)
    # final_hidden [bs, seq_len, hidden_dim*2*num_layers]
    return final_hidden

In [12]:
class LinearAttention(nn.Module):

  def __init__(self, emb_dim):
    super().__init__()
    self.linear = nn.Linear(emb_dim, 1)

  def forward(self, question_emb, question_mask, stacked_bi_lstm_score):
    # question_emb [bs, seq_len, emb_dim]
    # question_mask [bs, seq_len]
    # stacked_bi_lstm_score [bs, seq_len, hidden_dim*2*num_layers]
    question_emb = self.linear(question_emb)
    # question_emb [bs, seq_len, 1]
    question_emb = question_emb.squeeze(2)
    # question_emb [bs, seq_len]
    question_emb = question_emb.masked_fill(question_mask == 0, -float('inf'))
    alphas = F.softmax(question_emb, dim=1)
    # alphas [bs, seq_len]
    alphas = alphas.unsqueeze(1)
    # alphas [bs, 1, seq_len]

    weighted_average = torch.bmm(alphas, stacked_bi_lstm_score)
    # weighted_average [bs, 1, hidden_dim*2*num_layers]
    weighted_average = weighted_average.squeeze(1)
    # weighted_average [bs, hidden_dim*2*num_layers]
    return weighted_average

In [13]:
class BiLinearAttention(nn.Module):

  def __init__(self, question_dim, context_dim):
    super().__init__()
    self.context_proj = nn.Linear(question_dim, context_dim)

  def forward(self, context, question, context_mask):
    # context [bs, seq_len, hidden_dim_ctx*2*num_layers]
    # question [bs, hidden_dim_qtn*2*num_layers]
    question = self.context_proj(question)
    # question [bs, hidden_dim_ctx*2*num_layers]
    question = question.unsqueeze(2)
    # question [bs, hidden_dim_ctx*2*num_layers, 1]
    score = torch.bmm(context, question)
    # score [bs, seq_len, 1]
    score = score.squeeze(2)
    # score [bs, seq_len]
    score = score.masked_fill(context_mask == 0, -float('inf'))
    # score = F.softmax(score, dim=1)
    return score

In [14]:
class DrQA(nn.Module):
  
  def __init__(self, voc_size, emb_dim, hidden_dim, num_layers, dropout, device, pad_idx):
    super().__init__()
    self.device = device
    self.context_embedding = nn.Embedding(voc_size, emb_dim, padding_idx=pad_idx)
    self.question_embedding = nn.Embedding(voc_size, emb_dim, padding_idx=pad_idx)
    
    self.context_bilstm = StackedBiLSTM(emb_dim * 2, hidden_dim, num_layers, dropout)
    self.question_bilstm = StackedBiLSTM(emb_dim, hidden_dim, num_layers, dropout)

    self.align_embedding = AlignParagraphLayer(emb_dim)
    self.linear_attn = LinearAttention(emb_dim)

    self.start_bilinear_attn = BiLinearAttention(hidden_dim * num_layers * 2, hidden_dim * num_layers * 2)
    self.end_bilinear_attn = BiLinearAttention(hidden_dim * num_layers * 2, hidden_dim * num_layers * 2)

  def _create_mask(self, text_lens):
    bs = text_lens.size(0)
    max_seq_len = torch.max(text_lens)
    mask = torch.ByteTensor(bs, max_seq_len).fill_(0)
    for i in range(bs):
      mask[i, :text_lens[i]] = 1
    return mask

  def forward(self, context, question, context_lens, question_lens):
    context_emb = self.context_embedding(context)
    question_emb = self.context_embedding(question)

    context_mask = self._create_mask(context_lens).to(self.device)
    question_mask = self._create_mask(question_lens).to(self.device)

    aligned = self.align_embedding(context_emb, question_emb, question_mask)
    context_bilstm_input = torch.cat([context_emb, aligned], dim=2)

    context_bilstm_output = self.context_bilstm(context_bilstm_input, context_lens)
    
    question_bilstm_output = self.question_bilstm(question_emb, question_lens)
    question_weighted_linear_attn = self.linear_attn(question_emb, question_mask, question_bilstm_output)

    start_prediction = self.start_bilinear_attn(context_bilstm_output, question_weighted_linear_attn, context_mask)
    end_prediction = self.end_bilinear_attn(context_bilstm_output, question_weighted_linear_attn, context_mask)

    return start_prediction, end_prediction

In [24]:
# voc_size, emb_dim, hidden_dim, num_layers, dropout, device, pad_idx

config = {
    'voc_size': len(TEXT.vocab),
    'emb_dim': 256,
    'hidden_dim': 256,
    'num_layers': 3,
    'dropout': 0.3,
    'device': device,
    'pad_idx': TEXT.vocab.stoi[TEXT.pad_token]
}

drqa = DrQA(**config).to(device)
start_criterion = nn.CrossEntropyLoss(ignore_index=config['pad_idx']).to(device)
end_criterion = nn.CrossEntropyLoss(ignore_index=config['pad_idx']).to(device)
optimizer = torch.optim.Adam(drqa.parameters())

In [25]:
def train_epoch(model, iterator, start_criterion, end_criterion, optimizer, device):
  model.train()
  error = 0
  for batch in iterator:

    optimizer.zero_grad()

    context, context_lens = batch.story
    question, question_lens = batch.query
    answer, answer_lens = batch.answer
    context_lens = context_lens.cpu()
    question_lens = question_lens.cpu()

    answer_indexes = answer2span(answer.cpu().numpy(), context.cpu().numpy())
    answer_indexes = np.array(answer_indexes)
    start_target = torch.LongTensor(answer_indexes[:, 0]).to(device)
    end_target = torch.LongTensor(answer_indexes[:, 1]).to(device)

    start, end = model(context, question, context_lens, question_lens)

    loss_start = start_criterion(start, start_target)
    loss_end = end_criterion(end, end_target)

    loss = loss_start + loss_end
    loss.backward()

    torch.nn.utils.clip_grad_norm_(drqa.parameters(), 10)

    optimizer.step()

    error += loss.detach().cpu().item()
  return error / len(iterator)

def valid_epoch(model, iterator, start_criterion, end_criterion, device):
  model.eval()
  error = 0
  with torch.no_grad():
    for batch in iterator:

      context, context_lens = batch.story
      question, question_lens = batch.query
      answer, answer_lens = batch.answer
      context_lens = context_lens.cpu()
      question_lens = question_lens.cpu()

      answer_indexes = answer2span(answer.cpu().numpy(), context.cpu().numpy())
      answer_indexes = np.array(answer_indexes)
      start_target = torch.LongTensor(answer_indexes[:, 0]).to(device)
      end_target = torch.LongTensor(answer_indexes[:, 1]).to(device)

      start, end = model(context, question, context_lens, question_lens)

      loss_start = start_criterion(start, start_target)
      loss_end = end_criterion(end, end_target)

      loss = loss_start + loss_end
      error += loss.detach().cpu().item()
  return error / len(iterator)

In [26]:
def predict(model, question, context, TEXT, device):
  model.eval()

  question_tokens = [TEXT.init_token] + question.split() + [TEXT.eos_token]
  context_tokens = [TEXT.init_token] + context.split() + [TEXT.eos_token]

  question_ids = [TEXT.vocab.stoi[token] for token in question_tokens]
  context_ids = [TEXT.vocab.stoi[token] for token in context_tokens]

  question_tensor = torch.LongTensor([question_ids]).to(device)
  context_tensor = torch.LongTensor([context_ids]).to(device)

  question_lens = torch.LongTensor([len(question_tokens)])
  context_lens = torch.LongTensor([len(context_tokens)])

  with torch.no_grad():
    start_pred, end_pred = model(context_tensor, question_tensor, context_lens, question_lens)
  
  start_pred = start_pred.squeeze(0)
  end_pred = end_pred.squeeze(0)

  start_pos = start_pred.detach().cpu().numpy().argmax()
  end_pos = end_pred.detach().cpu().numpy().argmax()

  if end_pos < start_pos:
    start_pos = end_pos - 1

  answer = ' '.join(context_tokens[start_pos:end_pos])
  return answer

In [27]:
for i in range(20):
  train_loss = train_epoch(drqa, train_iterator, start_criterion, end_criterion, optimizer, device)
  valid_loss = valid_epoch(drqa, valid_iterator, start_criterion, end_criterion, device)
  print(f'Epoch: {i + 1}: Train loss: {train_loss} Valid loss: {valid_loss}')

Epoch: 1: Train loss: 7.900926649570465 Valid loss: 3.881363868713379
Epoch: 2: Train loss: 2.532179318368435 Valid loss: 1.7815632820129395
Epoch: 3: Train loss: 1.4049102514982224 Valid loss: 0.9920347929000854
Epoch: 4: Train loss: 1.0417472124099731 Valid loss: 1.0860261917114258
Epoch: 5: Train loss: 0.9558154530823231 Valid loss: 1.106911301612854
Epoch: 6: Train loss: 1.182784341275692 Valid loss: 1.2071187496185303
Epoch: 7: Train loss: 0.8513849787414074 Valid loss: 0.9979051351547241
Epoch: 8: Train loss: 0.8487857654690742 Valid loss: 1.059713363647461
Epoch: 9: Train loss: 0.8059243746101856 Valid loss: 1.0308994054794312
Epoch: 10: Train loss: 0.9687634482979774 Valid loss: 0.9979742765426636
Epoch: 11: Train loss: 1.2684020698070526 Valid loss: 1.0653152465820312
Epoch: 12: Train loss: 1.0686724931001663 Valid loss: 1.192121982574463
Epoch: 13: Train loss: 0.8451103568077087 Valid loss: 1.069831371307373
Epoch: 14: Train loss: 0.7999449875205755 Valid loss: 0.981432795524

In [28]:
index = 1
question = ' '.join(train_data.examples[index].query)
context = ' '.join(train_data.examples[index].story)

print('story:\t', context)
print('query:\t', question)
print('answer:\t', predict(drqa, question, context, TEXT, device))

story:	 mary moved to the bathroom . john went to the hallway . daniel went back to the hallway . sandra moved to the garden
query:	 where is daniel
answer:	 hallway
